<img src="../../img/land_use_clc.png" alt="" style="width: 100%; border-radius: 20px;"/>

## Objective
ToDo:

why did we create this notebook? why landuse?

## Scope

In this file we process a bird dataframe and merge it with the Corine Land Cover Dataframe to get the land use on the location of a bird sighting. With the functions given in this file it is possible to get the land use on the coordinate, to get the most common land use in a square or to get all land uses per percentage in a square around the coordinate.

The CLC was used because the LUCAS dataframe was used first and gave a much less detailed result. The LUCAS dataframe has points on a map which describe the land use whereas the CLC dataframe has polygons all over the map.

To get the land use with LUCAS the nearest point to the coordinate has to be selected. To do the same with CLC (altough much more precises) the coordinates of the bird dataframe are checked whether they are in any polygon. Then the land use of this polygon is used.

To get the most common land use within a square we create squares around each coordinate. Then each square is checked for the landuses within. Then the most common land uses gets returned.

The probably best solution to get the land use is to get the percentage of landuses in each square. This can be done by checking how much of each polygon is in each square. This is really compute heavy and this is also why there might be a better option:

To accomplish this another aspect of the CLC data is used. The CLC data also has points all over the map which can be used to get the land use. While those points also could be inaccurate if looking for the land use on a coordinate, those specific points inside of the square could be counted and calculated how much of each land use is in each square.

In [2]:
import geopandas as gpd
from shapely.geometry import Point, box
import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
df_path = 'D:\Simon\Documents\GP\data\datasets\selected_bird_species_with_grids_50km.csv'
df = pd.read_csv(df_path, index_col=0, low_memory=False)

In [4]:
df_path = "D:\Simon\Documents\GP\data\master_validata.csv"
df = pd.read_csv(df_path, index_col=0, low_memory=False)

In [5]:
df.tail(2)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent
417884,87058,92,Knäkente,2023-03-17,45.937733,7.863343,4503.0,100.0,0,50kmE4150N2500,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.027466,0.0,0.0
417885,87059,160,Gänsegeier,2023-06-28,45.931807,7.871133,4533.0,1.0,0,50kmE4150N2500,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001992,0.0,0.0,1.025582,0.0,0.0


In [14]:
clc_path = 'D:\\Simon\\Documents\\GP\\data\\util_files\\CLC2018\\U2018_CLC2018_V2020_20u1.shp'
clc2 = gpd.read_file(clc_path)

In [ ]:
clc['Code_18'].fillna(999, inplace=True)
clc['Code_18'] = clc['Code_18'].astype(int)
clc.drop(columns=['OBJECTID', 'Remark','Area_Ha','ID','Shape_Leng','Shape_Area'], axis=1, inplace=True)

In [6]:
clc_path = r'D:\\Simon\\Documents\\GP\\data\\util_files\\u2018_clc2018_v2020_20u1_geoPackage\\DATA\U2018_CLC2018_V2020_20u1.gpkg'
clc = gpd.read_file(clc_path)

In [7]:
clc['Code_18'].fillna(999, inplace=True)
clc['Code_18'] = clc['Code_18'].astype(int)
clc.drop(columns=['Remark','Area_Ha','ID'], axis=1, inplace=True)

In [8]:
clc.crs

<Projected CRS: EPSG:3035>
Name: ETRS89-extended / LAEA Europe
Axis Info [cartesian]:
- Y[north]: Northing (metre)
- X[east]: Easting (metre)
Area of Use:
- name: Europe - European Union (EU) countries and candidates. Europe - onshore and offshore: Albania; Andorra; Austria; Belgium; Bosnia and Herzegovina; Bulgaria; Croatia; Cyprus; Czechia; Denmark; Estonia; Faroe Islands; Finland; France; Germany; Gibraltar; Greece; Hungary; Iceland; Ireland; Italy; Kosovo; Latvia; Liechtenstein; Lithuania; Luxembourg; Malta; Monaco; Montenegro; Netherlands; North Macedonia; Norway including Svalbard and Jan Mayen; Poland; Portugal including Madeira and Azores; Romania; San Marino; Serbia; Slovakia; Slovenia; Spain including Canary Islands; Sweden; Switzerland; Türkiye (Turkey); United Kingdom (UK) including Channel Islands and Isle of Man; Vatican City State.
- bounds: (-35.58, 24.6, 44.83, 84.73)
Coordinate Operation:
- name: Europe Equal Area 2001
- method: Lambert Azimuthal Equal Area
Datum: Eur

In [20]:
clc.head(2)

,Code_18,geometry
0,111,"MULTIPOLYGON (((1917182.160 943608.860, 191714..."
1,111,"MULTIPOLYGON (((1953122.840 950507.440, 195311..."


In [28]:
print(clc.iloc[0].geometry)

MULTIPOLYGON (((1917182.1600000001 943608.8599999994, 1917144.8200000003 943692.0999999996, 1917124.1099999994 943729.4399999995, 1917095.2300000004 943742.2400000002, 1917091.7599999998 943744.5, 1917061.0500000007 943766.75, 1917033.5299999993 943769.3300000001, 1917017.0099999998 943769.6999999993, 1917005.4900000002 943769.9499999993, 1916974.3100000005 943750.25, 1916955.5 943691.8900000006, 1916950.6999999993 943645.5, 1916943.1899999995 943582.1400000006, 1916939.1999999993 943536.5800000001, 1916937.4000000004 943531.7699999996, 1916942.5299999993 943525.2799999993, 1916858.67 943330.4700000007, 1916828.0999999996 943118.2400000002, 1916812.1400000006 943106.1600000001, 1916809.58 943104.2599999998, 1916801.08 943098.0899999999, 1916790.58 943090.4700000007, 1916763.0700000003 943084.5, 1916735.7699999996 943085.6600000001, 1916696.8599999994 943117.8000000007, 1916682.4399999995 943138.0500000007, 1916660.8800000008 943176.1799999997, 1916657.8900000006 943180.7599999998, 1916

In [26]:
clc_4326 = clc.to_crs("EPSG:4326")

In [29]:
print(clc_4326.iloc[0].geometry)

MULTIPOLYGON (((-14.352314025456243 28.056594856015785, -14.352883006054203 28.057217531584072, -14.353176672749157 28.057485731324494, -14.353488709652064 28.057516506495894, -14.35352799402502 28.057526514880713, -14.353881248962097 28.057634743172787, -14.354154718544239 28.057580011653567, -14.354315953505006 28.057536854107724, -14.354428368571137 28.057506688547083, -14.354682046359816 28.05724694960394, -14.354719709618982 28.056684058883146, -14.35465113142415 28.05626512920942, -14.354566727550445 28.055690270053244, -14.354492352494361 28.055280868106323, -14.354497880077203 28.055233773602172, -14.35443198764748 28.05519145666469, -14.354762201952587 28.053253303358, -14.35453206570966 28.051312539312367, -14.354656954383648 28.051162139339667, -14.354677080057945 28.05113834403019, -14.354744247526817 28.051060547460377, -14.354827223364039 28.05096446116177, -14.355079354639503 28.050835027699776, -14.355347144956001 28.050768500388305, -14.355804491826405 28.0509401363055

## Get landuse on coordinate

In [ ]:
df.drop(columns=['land_use_coord', 'land_use_coord_numeric', 'geometry'], inplace=True)

In [ ]:
df.tail(5)

In [30]:
geometry = [Point(lon, lat) for lon, lat in zip(df['coord_lon'], df['coord_lat'])]
#gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:3035")
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

merged_gdf = gpd.sjoin(gdf, clc_4326, how="left", predicate="within")

merged_gdf.drop(columns=['geometry','index_right'], axis=1, inplace=True)

In [31]:
merged_gdf.tail(5)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,Code_18
417881,87055,8,Haubentaucher,2023-01-03,46.091253,7.858869,4310.0,2.0,0,50kmE4150N2550,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.756513,0.0,0.0,0.268142,0.0,0.0,332.0
417882,87056,397,Schwarzkehlchen,2023-03-12,46.101257,7.715938,4487.0,6.0,0,50kmE4100N2550,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.668966,0.0,0.0,0.355505,0.0,0.0,332.0
417883,87057,396,Braunkehlchen,2023-05-18,45.937974,7.867263,4496.0,2.0,10,50kmE4150N2500,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038580,0.0,0.0,0.988882,0.0,0.0,335.0
417884,87058,92,Knäkente,2023-03-17,45.937733,7.863343,4503.0,100.0,0,50kmE4150N2500,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.027466,0.0,0.0,335.0
417885,87059,160,Gänsegeier,2023-06-28,45.931807,7.871133,4533.0,1.0,0,50kmE4150N2500,distribution (coord),1,ch,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.001992,0.0,0.0,1.025582,0.0,0.0,335.0


In [32]:
nan_rows = merged_gdf[merged_gdf['Code_18'].isna()]
len(nan_rows)

4

In [35]:
nan_rows.head(5)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,Code_18
280243,280244,52,Singschwan,2023-03-25,54.745360,14.252600,0.0,14.0,NaN,50kmE4550N3500,NaN,0,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330499,330557,158,Seeadler,2023-01-28,54.049885,7.059010,409.0,1.0,NaN,50kmE4100N3400,distribution (coord),1,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330567,330625,52,Singschwan,2023-01-08,54.078992,7.178066,62.0,4.0,NaN,50kmE4100N3400,distribution (coord),1,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
330586,330644,52,Singschwan,2023-03-08,54.837358,6.977549,-2.0,3.0,NaN,50kmE4100N3500,distribution (coord),1,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
merged_gdf['Code_18'].fillna(999, inplace=True)

In [36]:
clc_code_to_landuse = {
    111: 'urban', # 1
    112: 'urban',
    121: 'urban',
    122: 'urban',
    123: 'urban',
    124: 'urban',
    131: 'industrial', # 2
    132: 'industrial',
    133: 'industrial',
    141: 'urban', 
    142: 'urban',
    211: 'agriculture', # 3
    212: 'agriculture',
    213: 'agriculture',
    221: 'agriculture',
    222: 'agriculture',
    223: 'agriculture',
    231: 'agriculture',
    241: 'agriculture',
    242: 'agriculture',
    243: 'agriculture',
    244: 'agriculture',
    311: 'forest', # 4
    312: 'forest',
    313: 'forest',
    321: 'grassland', # 5 
    322: 'grassland',
    323: 'mediterranean_vegetation', # 6 
    324: 'shrubland', # 7 
    331: 'coastal', # 8 
    332: 'rocky_area',# 9
    333: 'sparsley_vegetated', # 10
    334: 'burnt_area', # 11
    335: 'glacier', # 12
    411: 'wetlands', # 13
    412: 'wetlands',
    421: 'wetlands',
    422: 'wetlands',
    423: 'wetlands',
    511: 'water', # 14
    512: 'water',
    521: 'water',
    522: 'water',
    523: 'water',
    990: 'unclassified_land', # 15
    995: 'unclassified_water', # 16
    999: 'unclassified' # 17
}
merged_gdf['land_use_coord'] = merged_gdf['Code_18'].map(clc_code_to_landuse)
merged_gdf.drop(columns=['Code_18'], axis=1, inplace=True)

In [37]:
merged_gdf.head(5)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord
0,1,183,Alpenschneehuhn,2023-01-20,47.42037,10.35089,2057.0,1.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.241985,0.0,0.0,0.224493,0.533521,0.0,0.0,0.0,0.0,sparsley_vegetated
1,2,183,Alpenschneehuhn,2023-01-20,47.42092,10.35499,2009.0,1.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.209926,0.0,0.0,0.179440,0.610624,0.0,0.0,0.0,0.0,sparsley_vegetated
2,3,183,Alpenschneehuhn,2023-01-31,47.41393,10.34670,1922.0,2.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.946052,0.0,0.0,0.000000,0.054069,0.0,0.0,0.0,0.0,grassland
3,4,183,Alpenschneehuhn,2023-02-28,47.41693,10.34055,1902.0,3.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.724098,0.0,0.0,0.024907,0.251060,0.0,0.0,0.0,0.0,grassland
4,5,183,Alpenschneehuhn,2023-04-12,47.42830,10.39052,1629.0,1.0,NaN,50kmE4350N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.999851,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,grassland


In [39]:
landuse_to_numerical = {
    'urban': 1,
    'industrial': 2,
    'agriculture': 3,
    'forest': 4,
    'grassland': 5,
    'mediterranean_vegetation': 6,
    'shrubland': 7,
    'coastal': 8,
    'rocky_area': 9,
    'sparsley_vegetated': 10,
    'burnt_area': 11,
    'glacier': 12,
    'wetlands': 13,
    'water': 14,
    'unclassified_land': 15,
    'unclassified_water': 16,
    'unclassified': 17
}
merged_gdf['land_use_coord_numeric'] = merged_gdf['land_use_coord'].map(landuse_to_numerical)

In [40]:
merged_gdf.head(5)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
0,1,183,Alpenschneehuhn,2023-01-20,47.42037,10.35089,2057.0,1.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.241985,0.0,0.0,0.224493,0.533521,0.0,0.0,0.0,0.0,sparsley_vegetated,10
1,2,183,Alpenschneehuhn,2023-01-20,47.42092,10.35499,2009.0,1.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.209926,0.0,0.0,0.179440,0.610624,0.0,0.0,0.0,0.0,sparsley_vegetated,10
2,3,183,Alpenschneehuhn,2023-01-31,47.41393,10.34670,1922.0,2.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.946052,0.0,0.0,0.000000,0.054069,0.0,0.0,0.0,0.0,grassland,5
3,4,183,Alpenschneehuhn,2023-02-28,47.41693,10.34055,1902.0,3.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.724098,0.0,0.0,0.024907,0.251060,0.0,0.0,0.0,0.0,grassland,5
4,5,183,Alpenschneehuhn,2023-04-12,47.42830,10.39052,1629.0,1.0,NaN,50kmE4350N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.999851,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,grassland,5


In [41]:
merged_gdf.value_counts('land_use_coord', dropna=False)

land_use_coord
agriculture           185674
water                  94864
forest                 39807
urban                  32833
wetlands               32014
grassland              23056
industrial              3535
shrubland               2730
sparsley_vegetated      1693
rocky_area               894
coastal                  661
glacier                  105
burnt_area                16
unclassified               4
Name: count, dtype: int64

In [44]:
merged_gdf.land_use_coord.isnull().sum()

0

In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_on_coord_validata_ch.csv'
merged_gdf.to_csv(df_path)

In [ ]:
# write to other csvs
df_path = "D:\Simon\Documents\GP\data\master_validata.csv"
merged_gdf.to_csv(df_path)

### Create squares around each coordinate
This is to determine the most common land use and also the land use in percentage within a square

In [ ]:
# Convert 1km to degrees
km_to_degrees = 1 / 111.0

# Define the size of the square in degrees
square_size_horizontal = 1 * km_to_degrees
square_size_vertical = square_size_horizontal * 1.6667  # To make it a square

# Create a GeoDataFrame with square polygons around each point
geometry = [Point(lon, lat) for lon, lat in zip(df['coord_lon'], df['coord_lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:3035")

# Create squares around each point
squares_gdf = gdf.copy()
squares_gdf['geometry'] = gdf['geometry'].apply(lambda point: box(
    point.x - square_size_vertical/2, 
    point.y - square_size_horizontal/2,
    point.x + square_size_vertical/2, 
    point.y + square_size_horizontal/2
))

In [ ]:
squares_gdf.tail(5)

In [ ]:
squares_gdf.crs

## Get land use percentage 
(better version - old oversion is below in the notebook)

In [ ]:
df.drop(columns=['urban_area_percent','industrial_area_percent','agriculture_area_percent','forest_area_percent','grassland_area_percent','shrubland_area_percent','coastal_area_percent','rocky_area_percent','sparsley_vegetated_area_percent','burnt_area_percent','glacier_area_percent','wetlands_area_percent','water_area_percent'], inplace=True)

In [ ]:
# execution time: 215m (8 Core CPU)
# crs_projected = "EPSG:4326"
crs_projected = "EPSG:3035"

# Transform your data to the projected CRS 
squares_gdf_projected = squares_gdf.to_crs(crs_projected)
clc_projected = clc.to_crs(crs_projected)

# Perform a spatial join between squares_gdf and clc
joined = gpd.sjoin(squares_gdf_projected, clc_projected, how="inner", predicate='intersects')

# Calculate the area of intersection 
joined['intersection_area'] = joined.apply(lambda x: x['geometry'].intersection(clc_projected.loc[x['index_right'], 'geometry']).area, axis=1)

# Group by the index of squares_gdf and Code_18, and sum the intersection areas 
grouped = joined.groupby([joined.index, 'Code_18'])['intersection_area'].sum()

# Calculate the square area - all squares have the same size so we can just take the area of the first square
total_area = squares_gdf_projected.geometry.area.loc[grouped.index.get_level_values(0)].iloc[0]

# Calculate the percentage of each land use type within each square 
percentage_land_use = grouped / total_area

# Unstack the grouped dataframe to get a dataframe where each row corresponds to a square and each column corresponds to a land use type
land_use_df = percentage_land_use.unstack(fill_value=0)

# Join the land use dataframe with squares_gdf 
result = pd.concat([squares_gdf_projected, land_use_df], axis=1)
result.drop(columns=['geometry'], inplace=True)

In [84]:
result.head(3)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
0,1,183,Alpenschneehuhn,2023-01-20,47.42037,10.35089,2057.0,1.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.241985,0.0,0.0,0.224493,0.533521,0.0,0.0,0.0,0.0,sparsley_vegetated,10
1,2,183,Alpenschneehuhn,2023-01-20,47.42092,10.35499,2009.0,1.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.209926,0.0,0.0,0.179440,0.610624,0.0,0.0,0.0,0.0,sparsley_vegetated,10
2,3,183,Alpenschneehuhn,2023-01-31,47.41393,10.34670,1922.0,2.0,NaN,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.946052,0.0,0.0,0.000000,0.054069,0.0,0.0,0.0,0.0,grassland,5


In [177]:
result = merged_gdf.copy()

In [ ]:
nan_rows = result[result[112].isna()]
len(nan_rows)

In [179]:
category_dict = {
    111: 'urban_area_percent',
    112: 'urban_area_percent',
    121: 'urban_area_percent',
    122: 'urban_area_percent',
    123: 'urban_area_percent',
    124: 'urban_area_percent',
    131: 'industrial_area_percent',
    132: 'industrial_area_percent',
    133: 'industrial_area_percent',
    141: 'urban_area_percent',
    142: 'urban_area_percent',
    211: 'agriculture_area_percent',
    212: 'agriculture_area_percent',
    213: 'agriculture_area_percent',
    221: 'agriculture_area_percent',
    222: 'agriculture_area_percent',
    223: 'agriculture_area_percent',
    231: 'agriculture_area_percent',
    241: 'agriculture_area_percent',
    242: 'agriculture_area_percent',
    243: 'agriculture_area_percent',
    244: 'agriculture_area_percent',
    311: 'forest_area_percent',
    312: 'forest_area_percent',
    313: 'forest_area_percent',
    321: 'grassland_area_percent',
    322: 'grassland_area_percent',
    323: 'mediterranean_vegetation_area_percent',
    324: 'shrubland_area_percent',
    331: 'coastal_area_percent',
    332: 'rocky_area_percent',
    333: 'sparsley_vegetated_area_percent',
    334: 'burnt_area_percent',
    335: 'glacier_area_percent',
    411: 'wetlands_area_percent',
    412: 'wetlands_area_percent',
    421: 'wetlands_area_percent',
    422: 'wetlands_area_percent',
    423: 'wetlands_area_percent',
    511: 'water_area_percent',
    512: 'water_area_percent',
    521: 'water_area_percent',
    522: 'water_area_percent',
    523: 'water_area_percent',
    990: 'unclassified_land_area_percent',
    995: 'unclassified_water_area_percent',
    999: 'unclassified_area_percent'
}

category_df = result.copy()
for code, category in category_dict.items():
    if code in result.columns:
        if category not in category_df.columns:
            category_df[category] = result[code]
        else:
            category_df[category] += result[code]


columns_to_drop = [col for col in category_dict.keys() if col in category_df.columns]
category_df.drop(columns=columns_to_drop, inplace=True)


In [180]:
nan_rows = category_df[category_df['urban_area_percent'].isna()]
print(nan_rows.shape)
nan_rows.head(4)

(4, 28)


,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric
280243,280244,52,Singschwan,2023-03-25,54.745360,14.252600,0.0,14.0,NaN,50kmE4550N3500,NaN,0,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclassified,17
330499,330557,158,Seeadler,2023-01-28,54.049885,7.059010,409.0,1.0,NaN,50kmE4100N3400,distribution (coord),1,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclassified,17
330567,330625,52,Singschwan,2023-01-08,54.078992,7.178066,62.0,4.0,NaN,50kmE4100N3400,distribution (coord),1,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclassified,17
330586,330644,52,Singschwan,2023-03-08,54.837358,6.977549,-2.0,3.0,NaN,50kmE4100N3500,distribution (coord),1,de,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unclassified,17


In [181]:
category_df.iloc[330499]

id_validata                                      330557
id_species                                          158
name_species                                   Seeadler
date                                         2023-01-28
coord_lat                                     54.049885
coord_lon                                       7.05901
altitude                                          409.0
total_count                                         1.0
atlas_code                                          NaN
eea_grid_id                              50kmE4100N3400
edit_type                          distribution (coord)
edited                                                1
country                                              de
urban_area_percent                                  NaN
industrial_area_percent                             NaN
agriculture_area_percent                            NaN
forest_area_percent                                 NaN
grassland_area_percent                          

In [182]:
cols_to_check = list(set(category_df.columns).intersection(set(category_dict.values())))

all_nan_rows = category_df[cols_to_check].isna().all(axis=1)

category_df.loc[all_nan_rows, cols_to_check] = 0

category_df.loc[all_nan_rows, 'unclassified_area_percent'] = 1

category_df.loc[~all_nan_rows, 'unclassified_area_percent'] = 0

In [183]:
category_df.iloc[330499]

id_validata                                      330557
id_species                                          158
name_species                                   Seeadler
date                                         2023-01-28
coord_lat                                     54.049885
coord_lon                                       7.05901
altitude                                          409.0
total_count                                         1.0
atlas_code                                          NaN
eea_grid_id                              50kmE4100N3400
edit_type                          distribution (coord)
edited                                                1
country                                              de
urban_area_percent                                  0.0
industrial_area_percent                             0.0
agriculture_area_percent                            0.0
forest_area_percent                                 0.0
grassland_area_percent                          

In [168]:
nan_rows = category_df[category_df['unclassified_area_percent'].isna()]
print(nan_rows.shape)
nan_rows.head(2)

(0, 29)


,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent


In [175]:
category_df.head(3)

,id_validata,id_species,name_species,date,coord_lat,coord_lon,altitude,total_count,atlas_code,eea_grid_id,edit_type,edited,country,urban_area_percent,industrial_area_percent,agriculture_area_percent,forest_area_percent,grassland_area_percent,shrubland_area_percent,coastal_area_percent,rocky_area_percent,sparsley_vegetated_area_percent,burnt_area_percent,glacier_area_percent,wetlands_area_percent,water_area_percent,land_use_coord,land_use_coord_numeric,unclassified_area_percent
0,1,183,Alpenschneehuhn,2023-01-20,47.42037,10.35089,2057.0,1.0,0,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.241985,0.0,0.0,0.224493,0.533521,0.0,0.0,0.0,0.0,sparsley_vegetated,10,0.0
1,2,183,Alpenschneehuhn,2023-01-20,47.42092,10.35499,2009.0,1.0,0,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.209926,0.0,0.0,0.179440,0.610624,0.0,0.0,0.0,0.0,sparsley_vegetated,10,0.0
2,3,183,Alpenschneehuhn,2023-01-31,47.41393,10.34670,1922.0,2.0,0,50kmE4300N2700,NaN,0,de,0.0,0.0,0.0,0.0,0.946052,0.0,0.0,0.000000,0.054069,0.0,0.0,0.0,0.0,grassland,5,0.0


In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_percentage.csv'
category_df.to_csv(df_path)

In [174]:
# write to other csvs
df_path = 'D:\Simon\Documents\GP\data\master_validata.csv'
category_df.to_csv(df_path)

## Get most common landuse within square

In [ ]:
# execution time: 132m
merged_squares = gpd.sjoin(squares_gdf, clc, how="left", predicate="intersects")

In [ ]:
merged_squares.head(1)

In [ ]:
merged_squares.reset_index(drop=True, inplace=True)

In [ ]:
common_land_use = merged_squares.groupby(merged_squares.index)['Code_18'].agg(lambda x: x.value_counts().idxmax() if not x.empty and not x.value_counts().empty else None)

common_land_use_df = pd.DataFrame({'Code_18_sq': common_land_use.values}, index=common_land_use.index)

In [ ]:
common_land_use_df.head(2)

In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_most_common.csv'
common_land_use_df.to_csv(df_path)

## Get land use percentage 

In [ ]:
squares_with_clc = gpd.sjoin(clc, squares_gdf, how='inner', predicate='intersects')

# Calculate area of intersection between squares and CLC polygons for each Code_18 within each square
intersection_areas = squares_with_clc.groupby(['index_right', 'Code_18'])['geometry'].apply(lambda x: x.area)
total_areas = intersection_areas.groupby('index_right').sum()

# Calculate percentages of each Code_18 within each square
results = {}
for square_id, group in intersection_areas.groupby(level=0):
    percentages = {}
    for code_18, area in group.items():
        percentages[code_18[1]] = (area / total_areas[square_id])
    results[square_id] = percentages

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results).T.fillna(0)

In [ ]:
results_df.head(2)

In [ ]:
# Reset the index of results_df to make it easier to merge
results_df.reset_index(inplace=True)

# Merge the original DataFrame 'df' with results_df on the common index or a column
merged_df = pd.merge(df, results_df, left_index=True, right_on='index')

In [ ]:
merged_df.head(2)

In [ ]:
# write
df_path = 'D:\Simon\Documents\GP\data\datasets\land_use\land_use_percentage.csv'
merged_df.to_csv(df_path)

In [ ]:
common_land_use_df.head(3)

In [ ]:
# Merge the common land use DataFrame with the original DataFrame based on the index
merged_gdf = merged_squares.merge(common_land_use_df, left_index=True, right_index=True)
merged_gdf.drop(columns=['geometry'], axis=1, inplace=True)

In [ ]:
merged_gdf.head(3)

In [ ]:
clc_code_to_numerical_label = {
    111: 'urban', # 1
    112: 'urban',
    121: 'urban',
    122: 'urban',
    123: 'urban',
    124: 'urban',
    131: 'industrial', # 2
    132: 'industrial',
    133: 'industrial',
    141: 'urban', 
    142: 'urban',
    211: 'agriculture', # 3
    212: 'agriculture',
    213: 'agriculture',
    221: 'agriculture',
    222: 'agriculture',
    223: 'agriculture',
    231: 'agriculture',
    241: 'agriculture',
    242: 'agriculture',
    243: 'agriculture',
    244: 'agriculture',
    311: 'forest', # 4
    312: 'forest',
    313: 'forest',
    321: 'grassland', # 5 
    322: 'grassland',
    323: 'mediterranean_vegetation', # 6 
    324: 'shrubland', # 7 
    331: 'coastal', # 8 
    332: 'rocky_area',# 9
    333: 'sparsley_vegetated', # 10
    334: 'burnt_area', # 11
    335: 'glacier', # 12
    411: 'wetlands', # 13
    412: 'wetlands',
    421: 'wetlands',
    422: 'wetlands',
    423: 'wetlands',
    511: 'water', # 14
    512: 'water',
    521: 'water',
    522: 'water',
    523: 'water',
    990: 'unclassified_land', # 15
    995: 'unclassified_water', # 16
    999: 'unclassified' # 17
}
merged_gdf['land_use_coord'] = merged_gdf['Code_18'].map(clc_code_to_numerical_label)
merged_gdf['land_use_sq'] = merged_gdf['Code_18_sq'].map(clc_code_to_numerical_label)

In [ ]:
merged_gdf.head(3)

In [ ]:
merged_gdf['atlas_code'].fillna(0, inplace=True)

In [ ]:
merged_gdf.drop(columns=['Code_18', 'Code_18_sq'], axis=1, inplace=True)

In [ ]:
df_path = 'D:\Simon\Documents\GP\data\datasets\selected_species_50km_luse.csv'
merged_gdf.to_csv(df_path)